# FIRDS ETL Process

This notebook is an example of how to extract, transform and load financial instruments data in the FIRDS database.

Data:
- financial instruments data from FIRDS database
- url: https://registers.esma.europa.eu/solr/esma_registers_firds_files/select?q=*&fq=publication_date:%5B2021-01-17T00:00:00Z+TO+2021-01-19T23:59:59Z%5D&wt=xml&indent=true&start=0&rows=100
- The data is extracted in the XML format and then transformed into a CSV file.
- file_type: DLTINS
- Extracted attributes:
    - FinInstrmGnlAttrbts.Id
    - FinInstrmGnlAttrbts.FullNm
    - FinInstrmGnlAttrbts.ClssfctnTp
    - FinInstrmGnlAttrbts.CmmdtyDerivInd
    - FinInstrmGnlAttrbts.NtnlCcy
    - Issr
- Attributes obtained during the transformation process:
    - a_count
    - contains_a

In [1]:
import pandas as pd

from etl_processor import FIRDSExtractor, FIRDSLoader, FIRDSTransformer

firds_url = 'https://registers.esma.europa.eu/solr/esma_registers_firds_files/select?q=*&fq=publication_date:%5B2021-01-17T00:00:00Z+TO+2021-01-19T23:59:59Z%5D&wt=xml&indent=true&start=0&rows=100'
data_dir = 'data'
target_path = 'data/firds_golden.csv'

## 1. Extract - FIRDSExtractor

In [2]:
extractor = FIRDSExtractor(
    firds_url=firds_url,
    data_dir=data_dir,
)


async def main() -> None:
    """Run extract."""
    await extractor.arun()


await main()

100%|██████████| 4/4 [01:32<00:00, 23.25s/it]


## 2. Transform - FIRDSTransformer

In [3]:
transformer = FIRDSTransformer(
    data_dir=data_dir,
)

transformer.run()

## 3. load - FIRDSLoader

In [4]:
loader = FIRDSLoader(
    data_dir=data_dir,
    system='file',
    target_path=target_path,
)

loader.run()

C:\Users\ferna\OneDrive\Ambiente de Trabalho\etl-processor\etl_processor\load.py:88: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(self.firds_csv_path)


## Data Analysis

In [5]:
df = pd.read_csv(target_path)
df = df.sort_values(by='a_count', ascending=False)
df.head()

C:\Users\ferna\AppData\Local\Temp\ipykernel_21308\1785129476.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(target_path)


,FinInstrmGnlAttrbts.Id,FinInstrmGnlAttrbts.FullNm,FinInstrmGnlAttrbts.ClssfctnTp,FinInstrmGnlAttrbts.CmmdtyDerivInd,FinInstrmGnlAttrbts.NtnlCcy,Issr,a_count,contains_a
641830,XS2256949749,"ABERTIS INFRAESTRUCTURAS FINANCE B.V. €1,250,0...",DBFXQB,False,EUR,5493007WHKI5H75YJ358,14.0,True
665688,GB00BJ345J52,"HSBC BANK PLC Issue of 3,000 Volatility Disper...",RWITCE,False,USD,MP6I5ZYZBEU3UXPYFY54,11.0,True
45413,GB00BJ345J52,"HSBC BANK PLC Issue of 3,000 Volatility Disper...",RWITCE,False,USD,MP6I5ZYZBEU3UXPYFY54,11.0,True
365276,DE000HV4J2M3,Cash Collect Worst of con effetto memoria Quan...,DEEVRB,False,EUR,2ZCNRR8UK83OBTEK2170,10.0,True
991424,DE000MF72273,Faktor 2x Short Canadian National Railway emit...,RFSTPA,False,EUR,4PQUHN3JPFGFNF3BB653,10.0,True
